In [1]:
from transformers import pipeline
from datasets import load_dataset, Dataset
import pandas as pd

/home/sanka/anaconda3/envs/tf-env-ultimate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-20 14:27:31.531896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 14:27:32.756214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df11 = pd.read_parquet('volttronlog_mass.parquet')
df12= pd.read_parquet('540log_mass.parquet')
df21 = pd.read_parquet('volttronlog.parquet')
df22= pd.read_parquet('540log.parquet')
df=pd.concat([df11,df12,df21,df22],ignore_index=True)
df['response']=df['response'].apply(lambda array: array[0] )
raw=Dataset.from_pandas(df)
raw1=raw.train_test_split(test_size=0.2)
print(raw1)

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt', 'context'],
        num_rows: 3407
    })
    test: Dataset({
        features: ['response', 'prompt', 'context'],
        num_rows: 852
    })
})


In [3]:
from transformers import AutoTokenizer


checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [4]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["response"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:

tokenized_data = raw1.map(preprocess_function, batched=True)


Map: 100%|██████████| 852/852 [00:00<00:00, 1049.49 examples/s]


In [6]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
print(tokenized_data['train'])

Dataset({
    features: ['response', 'prompt', 'context', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3407
})


In [10]:
import evaluate

rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [8]:

from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=1e-5, weight_decay_rate=0.001)

In [9]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

2024-06-20 14:27:44.819341: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31070 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2024-06-20 14:27:44.820050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31133 MB memory:  -> device: 1, name: Tesla V100-DGXS-32GB, pci bus id: 0000:08:00.0, compute capability: 7.0
2024-06-20 14:27:44.820645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31133 MB memory:  -> device: 2, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0e:00.0, compute capability: 7.0
2024-06-20 14:27:44.821208: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31133 MB memory:  -> device: 3, name: Tesla V100-DGXS-32GB, pci bus id

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"  # Specify the model you want to use
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the input prompt
prompt = (
    "Given the following system events, generate a detailed description of the system:\n\n"
    "1. Event: User Login, Timestamp: 2024-06-24 10:00:00, Software Agent: AuthService\n"
    "2. Event: Data Retrieval, Timestamp: 2024-06-24 10:05:00, Software Agent: DataService\n"
    "3. Event: Data Update, Timestamp: 2024-06-24 10:10:00, Software Agent: UpdateService\n"
    "\nDescribe the overall system behavior, the roles of the software agents involved, and the sequence of operations."
)

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the output
outputs = model.generate(inputs.input_ids, max_length=300, num_return_sequences=1)

# Decode the generated text
system_description = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(system_description)


KeyboardInterrupt: 